# Editing an existing tensor

[Simon Dobson](mailto:simon.dobson@~st-andrews.ac.uk)<br>
School of Computer Science, University of St Andrews, Scotland UK

In [1]:
import json
import pickle
from datetime import datetime, timedelta
from itertools import product, combinations
from functools import reduce
import numpy
import netCDF4 as nc
from pandas import DataFrame
from geopandas import GeoDataFrame, GeoSeries, read_file
from networkx import Graph

from sensor_placement import *

import matplotlib
%matplotlib inline
%config InlineBackend.figure_format = 'png'
matplotlib.rcParams['figure.dpi'] = 300
import matplotlib.pyplot as plt

from pyproj import CRS, Transformer, Geod
from geovoronoi import voronoi_regions_from_coords, points_to_coords
import folium
import folium.plugins
import shapely
from shapely.geometry import Point, Polygon, MultiPolygon, MultiPoint, shape
from shapely.ops import unary_union, voronoi_diagram

In [2]:
uk_grid_crs = CRS.from_string('EPSG:27700')   # UK national grid
latlon_crs = CRS.from_string('EPSG:4326')     # global Mercator (WGS 84)

proj = Transformer.from_crs(uk_grid_crs, latlon_crs)
proj_inv = Transformer.from_crs(latlon_crs, uk_grid_crs)

In [3]:
boundaries_filename = 'datasets/UK_BUC.geojson'
with open(boundaries_filename, 'r') as fh:
    counties_json = json.load(fh)

In [4]:
counties = GeoDataFrame({'county': map(lambda c: c['properties']['ctyua18nm'], counties_json['features']),
                         'geometry': map(lambda c: shape(c['geometry']), counties_json['features'])})

## Constructing a small Voronoi diagram

In [5]:
fife = counties[counties['county'] == 'Fife'].iloc[0]
fife_boundary = fife['geometry']
mid_fife = list(list(fife_boundary.centroid.coords)[0])
mid_fife.reverse()

In [6]:
sepa_filename='datasets/sepa_monthly_2017.nc'
sepa = nc.Dataset(sepa_filename)

In [7]:
sepastations = GeoDataFrame({'id': numpy.asarray(sepa['station']).astype(int),
                             'name':numpy.asarray(sepa['name']),
                             'east':numpy.asarray(sepa['x']).astype(int),
                             'north':numpy.asarray(sepa['y']).astype(int),
                             'longitude':numpy.asarray(sepa['long']).astype(float),
                             'latitude': numpy.asarray(sepa['lat']).astype(float)})
sepastations['geometry'] = sepastations.apply(lambda r: Point(r['longitude'], r['latitude']), axis=1)
sepastations.set_index('id', inplace=True)

/home/sd80/programming/sensor-placement/venv3/lib/python3.7/site-packages/pandas/core/dtypes/cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


In [8]:
sepastations

,name,east,north,longitude,latitude,geometry
id,,,,,,
15018,Abbey St Bathans,375000,662000,-2.387448,55.853298,POINT (-2.38745 55.85330)
234150,Aberlour,327000,843000,-3.205975,57.480217,POINT (-3.20598 57.48022)
115301,Affric Lodge,218000,823000,-5.005980,57.262726,POINT (-5.00598 57.26273)
234170,Alford,356000,817000,-2.717123,57.242302,POINT (-2.71712 57.24230)
115302,Allanfearn,271000,847000,-4.151564,57.499149,POINT (-4.15156 57.49915)
...,...,...,...,...,...,...
115660,Waulkmill Glen,252000,658000,-4.353628,55.794842,POINT (-4.35363 55.79484)
116008,Weisdale Mill,439000,1153000,-1.288604,60.259327,POINT (-1.28860 60.25933)
115250,Westhill,382000,806000,-2.291989,57.147568,POINT (-2.29199 57.14757)


In [9]:
fife_stations = sepastations[sepastations['geometry'].within(fife_boundary)]

In [10]:
fife_stations

,name,east,north,longitude,latitude,geometry
id,,,,,,
15198,Annfield,314000,686000,-3.371633,56.066402,POINT (-3.37163 56.06640)
501947,Baintown Rain-gauge,335000,703000,-3.042679,56.221016,POINT (-3.04268 56.22102)
338380,Cambo Sands,359000,712000,-2.650068,56.300880,POINT (-2.65007 56.30088)
15151,Fife Airport,324000,699000,-3.222449,56.181652,POINT (-3.22245 56.18165)
473550,Kinghorn Ecology Centre Rain Gauge,325000,687000,-3.193433,56.074623,POINT (-3.19343 56.07462)
15083,Newton of Falkland,326000,707000,-3.186269,56.254314,POINT (-3.18627 56.25431)
15070,Rossie Farm,325000,712000,-3.202718,56.299126,POINT (-3.20272 56.29913)
15155,Saline,301000,693000,-3.585959,56.122238,POINT (-3.58596 56.12224)
335620,St Monance,351000,701000,-2.782714,56.204784,POINT (-2.78271 56.20478)


In [11]:
# find the boundaries of the region of interest
x_min, y_min, x_max, y_max = fife_boundary.bounds

# how many kilometre squares is that?
xg_min, yg_min = proj_inv.transform(y_min, x_min)
xg_max, yg_max = proj_inv.transform(y_max, x_max)
es = int((xg_max - xg_min) / 1000) + 1
ns = int((yg_max - yg_min) / 1000) + 1

xs = numpy.linspace(x_min, x_max, num=es, endpoint=True)
ys = numpy.linspace(y_min, y_max, num=ns, endpoint=True)

In [12]:
fife_tensor = NNNI(fife_stations, fife_boundary, ys, xs)

In [13]:
fife_stations_voronoi = folium.Map(location=mid_fife, tiles="Stamen Terrain", zoom_start=10)

# add the stations
for i in fife_stations.index:
    s = fife_stations.loc[i]
    name, lon, lat = s['name'], s['longitude'], s['latitude']
    folium.Marker(location=(lat, lon),
                  tooltip=f'{i}: {name} ({lat:.2f}N, {lon:.2f}W)',
                  icon=folium.Icon(color='green', icon='cloud')).add_to(fife_stations_voronoi)
    
# add the Voronoi cells
cell_cmap = plt.get_cmap('tab10')
len_cmap = 10
div_cmap = 1 / len_cmap
def cmap(i):    
    def style(c):
        c = cell_cmap(div_cmap * (i % (len_cmap + 1)) + (div_cmap / 2))
        rgb = 'rgb({r}, {g}, {b})'.format(r=int(c[0] * 256), g=int(c[1] * 256), b=int(c[2] * 256))
        return {'fillColor': rgb, 'fillOpacity': 0.5, 'color': 'black'}
    return style
cells = fife_tensor.cells()
for i in range(len(cells)):
    folium.GeoJson(cells[i], style_function=cmap(i)).add_to(fife_stations_voronoi)

In [14]:
fife_stations_voronoi

In [15]:
rainfall = []
for i in fife_stations.index:
    j = list(sepa['station'][:]).index(i)
    rainfall.append(sepa['rainfall_amount'][1, j])

In [16]:
fife_interpolated_rainfall = fife_tensor.apply(numpy.array(rainfall), clipped=True)

In [19]:
rainfall_min = numpy.amin(fife_interpolated_rainfall)
rainfall_max = numpy.amax(fife_interpolated_rainfall)
rainfall_delta = 1 / (rainfall_max - rainfall_min)

fife_rainpoints = []
mask = fife_interpolated_rainfall.mask
for i in range(len(ys)):
    for j in range(len(xs)):
        if not mask[i, j]:
            fife_rainpoints.append([ys[i], xs[j], (fife_interpolated_rainfall[i, j] - rainfall_min) * rainfall_delta])

In [23]:
fife_stations_interpolation = folium.Map(location=mid_fife, tiles="Stamen Terrain", zoom_start=10)
folium.GeoJson(fife['geometry']).add_to(fife_stations_interpolation)

# add the stations
for i, s in fife_stations.iterrows():
    name, lon, lat = s['name'], s['longitude'], s['latitude']
    j = list(sepa['station'][:]).index(i)
    rain = sepa['rainfall_amount'][1, j]
    folium.Marker(location=(lat, lon),
                  tooltip=f'{i}: {name} ({lat:.2f}N, {lon:.2f}W) {rain:.2f}mm').add_to(fife_stations_interpolation)

# add the Voronoi cell boundaries
for c in fife_tensor.cells():
    folium.GeoJson(c, style_function=lambda f: {'fill': False, 'color': 'black'}).add_to(fife_stations_interpolation)
    
# add the heat map
_ = folium.plugins.HeatMap(data=fife_rainpoints, min_opacity=0.01, radius=40, blur=40).add_to(fife_stations_interpolation)

In [24]:
fife_stations_interpolation

## Removing a sensor